In [1]:
import cv2
import pickle
from datetime import datetime
import requests as http

base_url = "http://localhost/restfull-api-pembelajaran-master/"
face_cascade = cv2.CascadeClassifier('cascade/haarcascade_frontalface_default.xml')
cap = cv2.VideoCapture(0)
recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read("train.yml")

labels = {}
with open("labels.pickle", "rb") as f:
    labels = pickle.load(f)
    labels = {v: k for k, v in labels.items()}
    
while True:
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.5, minNeighbors=5)
    
    for (x, y, w, h) in faces:
        roi_gray = gray[y:y + h, x:x + w]
        roi_color = frame[y:y + h, x:x + w]
        
        id_, conf = recognizer.predict(roi_gray)
        if conf >= 45:
            font = cv2.QT_FONT_NORMAL
            name = labels[id_]
            dataAbsen = {
                'nama' : name,
                'status': 'Hadir',
                'tgl_presensi': str(datetime.now()),
            }
           
            
            color = (255, 0, 0)
            stroke = 2
            cv2.putText(frame, name, (x, y), font, 1, color, stroke, cv2.LINE_AA)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), (2))
    cv2.imshow('cap', frame)
    k = cv2.waitKey(20) & 0xff
    if k == ord ('q'):
        break
request = http.post(base_url + 'api/presensi/checkin', json=dataAbsen)
if request.status_code == 200:
    print("Berhasil absen!")
else:
    print("Gagal absen!")
cap.release()
cv2.destroyAllWindows()


error: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
